In [1]:
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from __future__ import print_function

import os
import subprocess

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
file = '20160301_TE_survey_cleaned.csv'

In [4]:
df = pd.read_csv(file)

In [5]:
df.columns.values

array(['ID', 'T(K)', 'Z*10^-4 reported', 'Resist(Ohm.cm)',
       'Seebeck(uV/K)', 'kappaZT', 'Unnamed: 6', 'Resist(400K)',
       'Seebeck(400K)', 'Pf(W/K^2/m)', 'ZT', 'kappa(W/mK)', 'x',
       'Formula', 'series', 'T_Max', 'family', 'Unnamed: 17',
       'Unnamed: 18', 'Conduct(S/cm)', 'Power_Factor*T_(W/mK)',
       'preparative_route', 'final_form', 'cell_volume(A^3)',
       'formula_units_per_cell', 'atoms_per_formula_unit',
       'total_atoms_per_unit_cell', 'average_atomic_volume',
       'ICSD_of_structure', 'temp of ICSD (K)', 'S^2', 'ke/ktotal',
       'space_group', 'number_symmetry_elements', 'Unnamed: 34',
       'Unnamed: 35'], dtype=object)

In [115]:
df['Seebeck(uV/K)'].describe()

count    1098.000000
mean      -40.155270
std       192.691410
min      -752.196000
25%      -163.359500
50%       -67.600000
75%        99.358325
max      1235.430000
Name: Seebeck(uV/K), dtype: float64

### For the decision tree we need to bin the target values, so that we create a mock classification problem.

In [116]:
df['bin'] = pd.cut(df['Seebeck(uV/K)'], 50) #Breaking the Seebeck column into 50
                                            #bins, this number is arbitraty.

In [117]:
df = df.sort_values(['bin'], ascending = True) #sort the dataframe by ascending order
                                                #of the bins.

In [118]:
df['bin'].head()

1098                     NaN
1031    (-754.184, -712.443]
328     (-672.691, -632.938]
1032    (-632.938, -593.186]
124     (-632.938, -593.186]
Name: bin, dtype: category
Categories (50, interval[float64]): [(-754.184, -712.443] < (-712.443, -672.691] < (-672.691, -632.938] < (-632.938, -593.186] ... (1076.42, 1116.172] < (1116.172, 1155.925] < (1155.925, 1195.677] < (1195.677, 1235.43]]

In [119]:
#Pulling the columns we're interested in using for our predictors.
reduced_df = df[['Resist(Ohm.cm)', 'T(K)', 'Seebeck(uV/K)', 'average_atomic_volume', 'space_group', 'bin']]

In [120]:
reduced_df.head()

,Resist(Ohm.cm),T(K),Seebeck(uV/K),average_atomic_volume,space_group,bin
1098,NaN,NaN,NaN,NaN,NaN,NaN
1031,2.918890,300.0,-752.196,13.5200,139.0,"(-754.184, -712.443]"
328,0.302535,300.0,-650.910,10.3485,62.0,"(-672.691, -632.938]"
1032,2.161640,400.0,-618.425,13.5200,139.0,"(-632.938, -593.186]"
124,0.460937,400.0,-600.164,10.3485,62.0,"(-632.938, -593.186]"


### I need to assign each bin a number.  I created a new column with these bin numbers.

In [121]:
def encode_target(df, target_column):
    """Add column to data that assigns integers to the binned Seebeck data"""
    df_mod = df.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod['Target'] = df_mod[target_column].replace(map_to_int)
    
    return (df_mod, targets)

In [122]:
df2, targets = encode_target(reduced_df, 'bin')

In [123]:
df2.head()

,Resist(Ohm.cm),T(K),Seebeck(uV/K),average_atomic_volume,space_group,bin,Target
1098,NaN,NaN,NaN,NaN,NaN,NaN,0
1031,2.918890,300.0,-752.196,13.5200,139.0,"(-754.184, -712.443]",1
328,0.302535,300.0,-650.910,10.3485,62.0,"(-672.691, -632.938]",2
1032,2.161640,400.0,-618.425,13.5200,139.0,"(-632.938, -593.186]",3
124,0.460937,400.0,-600.164,10.3485,62.0,"(-632.938, -593.186]",3


In [124]:
#Clean up the NA values
df3 = df2.dropna()

In [125]:
#Break data into x and y
df3_x = pd.DataFrame(df3[['Resist(Ohm.cm)', 'T(K)', 'average_atomic_volume', 'space_group']])
df3_y = pd.DataFrame(df3['Target'])
df3_reg_y = pd.DataFrame(df3['Seebeck(uV/K)'])

In [126]:
#this just makes it easy to call all x parameters later on.
features = list(df3_x.columns[:4])

In [127]:
print (features)

['Resist(Ohm.cm)', 'T(K)', 'average_atomic_volume', 'space_group']


In [128]:
#Split into test/train
x_train, x_test = train_test_split(df3_x, test_size=0.2)
y_train, y_test = train_test_split(df3_y, test_size=0.2)

In [129]:
#Fit the decision tree,
#1st iteration dt = DecisionTreeRegressor(min_samples_split = 20, random_state = 99)
#Predictor gets better when max depth is decreased, but you loose too much info after a certain point.
y_fit_cla = y_train['Target']
X_fit_cla = x_train[features]
dt_cla = DecisionTreeClassifier(max_depth = 10, random_state = 99)
dt_fit_cla = dt_cla.fit(X_fit_cla, y_fit_cla)

In [130]:
dt_cla_pred = dt_cla.predict(x_test[features])
score = accuracy_score(y_test, dt_pred)
print(score)

NameError: name 'dt_pred' is not defined

In [131]:
#Split into test/train
x_train_reg, x_test_reg = train_test_split(df3_x, test_size=0.2)
y_train_reg, y_test_reg = train_test_split(df3_reg_y, test_size=0.2)

In [132]:
#Fit the decision tree,
#1st iteration dt = DecisionTreeRegressor(min_samples_split = 20, random_state = 99)
#Predictor gets better when max depth is decreased, but you loose too much info after a certain point.
y_fit = y_train_reg['Seebeck(uV/K)']
X_fit = x_train[features]
dt = DecisionTreeRegressor(max_depth = 10, random_state = 99)
dt_fit = dt.fit(x_train_reg, y_train_reg)

In [133]:
#Check the predictor.
dt_pred = dt.predict(x_test[features])
score = dt.score(x_test_reg, y_test_reg)
print (score)

-0.3731405657918343


In [134]:
#Random Forest is a different call, lets  try this out.
rdt = RandomForestRegressor(random_state=1010, n_estimators=100, max_depth = 10)
rdt_fit = rdt.fit(x_train_reg[features], y_train_reg['Seebeck(uV/K)'])
score = rdt.score(x_test_reg, y_test_reg)
print (score)

-0.14139435059047512


In [135]:
#Random Forest is a different call, lets  try this out.
rdt = RandomForestClassifier(random_state=1010, n_estimators=100, max_depth = 10)
rdt_fit = rdt.fit(x_train[features], y_train['Target'])
score = rdt.score(x_test, y_test)
print (score)

0.07009345794392523


In [20]:
#I was following along with a demo and the author mentioned a way to visalize
#the tree.  I haven't been able to get this to work yet.
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [21]:
visualize_tree(dt, features)

In [136]:
def encode_columns(df, df_mod2, target_column):
    """Change unique column values to integers"""
    targets = df_mod2[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod2[target_column] = df_mod2[target_column].replace(map_to_int)
    
    return

In [137]:
df_mod2 = df.copy()
encode_columns(df, df_mod2, 'preparative_route')

In [138]:
df_mod2['preparative_route'].head()

1098    0
1031    1
328     2
1032    3
124     2
Name: preparative_route, dtype: int64

In [139]:
reduced_df2 = df_mod2[['T(K)', 'Resist(Ohm.cm)', 'Seebeck(uV/K)', 'Resist(400K)', 'Conduct(S/cm)', 'preparative_route', 
                    'cell_volume(A^3)', 'average_atomic_volume', 'space_group', 'number_symmetry_elements', 'bin']]

In [140]:
reduced_df3 = reduced_df2.dropna()

In [141]:
df4, targets = encode_target(reduced_df3, 'bin')

In [142]:
df4_x_1 = pd.DataFrame(df4[['T(K)', 'Resist(Ohm.cm)', 'Resist(400K)', 'Conduct(S/cm)', 'preparative_route', 
                    'cell_volume(A^3)', 'average_atomic_volume', 'space_group', 'number_symmetry_elements']])
df4_y_1 = pd.DataFrame(df4['Target'])
df4_reg_y_1 = pd.DataFrame(df4['Seebeck(uV/K)'])

In [143]:
x_train_1, x_test_1 = train_test_split(df4_x_1, test_size=0.2)
y_train_1, y_test_1 = train_test_split(df4_y_1, test_size=0.2)
y_train_reg_1, y_train_reg_2 = train_test_split(df4_reg_y_1, test_size=0.2)

In [144]:
features_1 = list(df4_x_1.columns[:9])

In [145]:
print (features_1)

['T(K)', 'Resist(Ohm.cm)', 'Resist(400K)', 'Conduct(S/cm)', 'preparative_route', 'cell_volume(A^3)', 'average_atomic_volume', 'space_group', 'number_symmetry_elements']


In [146]:
dt_1 = DecisionTreeClassifier(max_depth = 10, random_state = 99)
dt_fit_1 = dt_1.fit(x_train_1[features_1], y_train_1['Target'],)

In [147]:
dt_pred_1 = dt_fit_1.score(x_test_1, y_test_1)
print (dt_pred_1)

0.14705882352941177


In [148]:
rdt_1 = RandomForestRegressor(random_state=1010, n_estimators=100, max_depth = 10)
rdt_fit_1 = rdt_1.fit(x_train_1[features_1], y_train_reg_1['Seebeck(uV/K)'])

In [149]:
rdt_pred_1 = rdt_fit_1.score(x_test_1, y_train_reg_2)
print (rdt_pred)

NameError: name 'rdt_pred' is not defined

In [26]:
import plotly
plotly.__version__

'2.4.1'

In [105]:
import plotly.plotly as py
import plotly.figure_factory as ff
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap
from bokeh.layouts import layout

In [28]:
elements.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,EA,standard state,bonding type,melting point,boiling point,density,metal,year discovered,group,period
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,-73.0,gas,diatomic,14.0,20.0,0.00009,nonmetal,1766,1,1
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,0.0,gas,atomic,NaN,4.0,0.00000,noble gas,1868,18,1
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,-60.0,solid,metallic,454.0,1615.0,0.54000,alkali metal,1817,1,2
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.57,90.0,45 (+2),NaN,...,0.0,solid,metallic,1560.0,2743.0,1.85000,alkaline earth metal,1798,2,2
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,-27.0,solid,covalent network,2348.0,4273.0,2.46000,metalloid,1807,13,2


In [103]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, Select
from bokeh.plotting import figure
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap, transform
from bokeh.palettes import RdBu

In [29]:
output_file("periodic.html")

periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = elements.copy()
df["atomic mass"] = df["atomic mass"].astype(str)
df["group"] = df["group"].astype(str)
df["period"] = [periods[x-1] for x in df.period]
df = df[df.group != "-"]
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]

cmap = {
    "alkali metal"         : "#a6cee3",
    "alkaline earth metal" : "#1f78b4",
    "metal"                : "#d93b43",
    "halogen"              : "#999d9a",
    "metalloid"            : "#e08d49",
    "noble gas"            : "#eaeaea",
    "nonmetal"             : "#f1d4Af",
    "transition metal"     : "#599d7A",
}

source = ColumnDataSource(df)

p = figure(title="Periodic Table (omitting LA and AC Series)", plot_width=1000, plot_height=450,
           tools="", toolbar_location=None,
           x_range=groups, y_range=list(reversed(periods)))

p.rect("group", "period", 0.95, 0.95, source=source, fill_alpha=0.6, legend="metal",
       color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

text_props = {"source": source, "text_align": "left", "text_baseline": "middle"}

x = dodge("group", -0.4, range=p.x_range)

r = p.text(x=x, y="period", text="symbol", **text_props)
r.glyph.text_font_style="bold"

r = p.text(x=x, y=dodge("period", 0.3, range=p.y_range), text="atomic number", **text_props)
r.glyph.text_font_size="8pt"

r = p.text(x=x, y=dodge("period", -0.35, range=p.y_range), text="name", **text_props)
r.glyph.text_font_size="5pt"

r = p.text(x=x, y=dodge("period", -0.2, range=p.y_range), text="atomic mass", **text_props)
r.glyph.text_font_size="5pt"

p.text(x=["3", "3"], y=["VI", "VII"], text=["LA", "AC"], text_align="center", text_baseline="middle")

p.add_tools(HoverTool(tooltips = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass}"),
    ("Type", "@metal"),
    ("CPK color", "$color[hex, swatch]:CPK"),
    ("Electronic configuration", "@{electronic configuration}"),
]))

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location ="top_center"

show(p)

In [254]:
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox, layout
from bokeh.models.widgets import Select

In [276]:
output_file("select.html")

temp_select = Select(title="Temperature:", value='300 K', options=["300 K", "400 K", "700 K", "1000 K"])


show(layout([[p],[temp_select]]))

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='904ac113-7f3c-43c5-bc54-3c4bd1a85f77', ...)]
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Dropdown(id='2bee0d6b-9bd6-4824-a2a3-a5ab31dfe7ff', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Dropdown(id='4897146f-7a74-44a1-8a40-d657bc732147', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Dropdown(id='49f47169-00ac-4bd2-b4d4-59440095ce48', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Dropdown(id='4aaea623-ae2b-44a6-a9f6-0c5709e50511', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='ef037ddd-890d-42b3-990c-cfe1c40aa88e', ...)
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout

In [284]:
output_file("select_prop.html")

def update_plot(attrname, old, new):
    prop = prop_select.value
    src = plt.scatter(elements['atomic number'], elements[value])
    return(src)


prop_select = Select(title="Property:", value='Electronegativity', options=['Electronegativity', 'atomic radius', 'EA'])

prop_select.on_change('value', update_plot)

show(layout([[src],[temp_select]]))

NameError: name 'src' is not defined

In [6]:
from temann.interpret import get_empirical_formula

In [7]:
periodic_df = df[['Seebeck(uV/K)', 'Pf(W/K^2/m)', 'Formula', 'T(K)']]

In [8]:
periodic_df.dropna()

,Seebeck(uV/K),Pf(W/K^2/m),Formula,T(K)
0,-462.967000,4.286770e-07,CaMnO3,300.0
1,-462.967000,4.286770e-07,CaMnO3,400.0
2,-470.651000,9.162900e-06,CaMnO3,700.0
3,-279.829000,4.411760e-05,CaMnO3,1000.0
4,-201.708000,8.822830e-05,Ca0.98La0.02MnO3,300.0
5,-201.708000,8.822830e-05,Ca0.98La0.02MnO3,400.0
6,-224.120000,7.721820e-05,Ca0.98La0.02MnO3,700.0
7,-190.181000,8.803530e-05,Ca0.98La0.02MnO3,1000.0
8,-123.586000,1.001180e-04,Ca0.96La0.04MnO3,300.0
9,-123.586000,1.001180e-04,Ca0.96La0.04MnO3,400.0


In [9]:
periodic_df1 = periodic_df[periodic_df['Seebeck(uV/K)'] < -150] 

In [10]:
periodic_df1 = periodic_df1.append(periodic_df[periodic_df['Seebeck(uV/K)'] > 150])

In [11]:
periodic_df1['T(K)'].unique()

array([ 300.,  400.,  700., 1000.,   nan])

In [12]:
T_300 = pd.DataFrame(periodic_df1[periodic_df1['T(K)'] == 300])

In [13]:
T_400 = pd.DataFrame(periodic_df1[periodic_df1['T(K)'] == 400])

In [14]:
T_700 = pd.DataFrame(periodic_df1[periodic_df1['T(K)'] == 700])

In [15]:
T_1000 = pd.DataFrame(periodic_df1[periodic_df1['T(K)'] == 1000])

In [16]:
def parse_multiple_formulas(df, target_df):
    for idx in range(len(df['Formula'])):
        parsed_formula = get_empirical_formula(df['Formula'].iloc[idx])
        target_df.append(parsed_formula)
        idx = idx + 1
    return (target_df)

In [17]:
parsed_T_300 = []
parse_multiple_formulas(periodic_df1, parsed_T_300)

[{'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 0.98, 'La': 0.02, 'Mn': 1, 'O': 3},
 {'Ca': 0.98, 'La': 0.02, 'Mn': 1, 'O': 3},
 {'Ca': 0.98, 'La': 0.02, 'Mn': 1, 'O': 3},
 {'Ca': 0.98, 'La': 0.02, 'Mn': 1, 'O': 3},
 {'Ca': 0.96, 'La': 0.04, 'Mn': 1, 'O': 3},
 {'Ca': 0.96, 'La': 0.04, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Ca': 1, 'Mn': 1, 'O': 3},
 {'Bi': 0.02, 'Ca': 0.98, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Bi': 0.02, 'Ca': 0.98, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Bi': 0.04, 'Ca': 0.96, 'Mn': 0.96, 'Nb': 0.04, 'O': 3},
 {'Bi': 0.1, 'Ca': 0.9, 'Mn': 0.9, 'Nb': 0.1, 'O': 3},
 {'Ca': 1, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Ca': 1, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Ca': 1, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Ca': 1, 'Mn': 0.98, 'Nb': 0.02, 'O': 3},
 {'Ca': 1, 'Mn': 0.98, 'O': 3, 'Ru': 0.02},
 {'Ca': 1, 'Mn': 0.98, 'O': 3, 'Ru': 0.02},
 {'Ca': 

In [18]:
parsed_T_300_df = pd.DataFrame(parsed_T_300)

In [19]:
parsed_T_300_df.head()

,Ag,Al,B,Ba,Bi,Ca,Ce,Co,Cr,Cs,...,Sr,Tb,Te,Ti,Tl,W,Y,Yb,Zn,Zr
0,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,0.98,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
results_T_300 = parsed_T_300_df[parsed_T_300_df.notnull()].count()

In [21]:
results_T_300_df = pd.DataFrame(results_T_300)

In [22]:
results_T_300_df = results_T_300_df.rename(columns = {0 : 'Count'})

In [23]:
results_T_300_df['symbol'] = results_T_300_df.index

In [24]:
results_T_300_df.head()

,Count,symbol
Ag,13,Ag
Al,62,Al
B,3,B
Ba,16,Ba
Bi,27,Bi


In [87]:
results_T_300_df['Count'].fillna(value=0, inplace=True)

In [85]:
print(results_T_300_df.loc[results_T_300_df['symbol'] == 'Rb'])

Empty DataFrame
Columns: [Count, symbol]
Index: []


In [ ]:
df1.merge(df2,how='left', left_on='Column1', right_on='ColumnA')

In [92]:
atomic_num_concat = elements.merge(results_T_300_df, how = 'left', on='symbol')

In [79]:
##atomic_num_concat = (pd.merge(elements, results_T_300_df, on='symbol'))

In [93]:
atomic_num_concat.replace(r'\s+', np.nan, regex=True)

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,bonding type,melting point,boiling point,density,metal,year discovered,group,period,counts,Count
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,diatomic,14.0,20.0,0.00009,nonmetal,1766,1,1,NaN,NaN
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,atomic,NaN,4.0,0.00000,NaN,1868,18,1,NaN,NaN
2,3,Li,Lithium,6.941,#CC80FF,NaN,0.98,134.0,NaN,182.0,...,metallic,454.0,1615.0,0.54000,NaN,1817,1,2,NaN,7.0
3,4,Be,Beryllium,9.012182,#C2FF00,NaN,1.57,90.0,NaN,NaN,...,metallic,1560.0,2743.0,1.85000,NaN,1798,2,2,NaN,NaN
4,5,B,Boron,10.811,#FFB5B5,NaN,2.04,82.0,NaN,NaN,...,NaN,2348.0,4273.0,2.46000,metalloid,1807,13,2,NaN,3.0
5,6,C,Carbon,12.0107,#909090,NaN,2.55,77.0,NaN,170.0,...,NaN,3823.0,4300.0,2.26000,nonmetal,Ancient,14,2,NaN,NaN
6,7,N,Nitrogen,14.0067,#3050F8,NaN,3.04,75.0,NaN,155.0,...,diatomic,63.0,77.0,0.00000,nonmetal,1772,15,2,NaN,NaN
7,8,O,Oxygen,15.9994,#FF0D0D,NaN,3.44,73.0,NaN,152.0,...,diatomic,55.0,90.0,0.00000,nonmetal,1774,16,2,0.0,260.0
8,9,F,Fluorine,18.9984032,#90E050,NaN,3.98,71.0,NaN,147.0,...,atomic,54.0,85.0,0.00000,halogen,1670,17,2,NaN,NaN
9,10,Ne,Neon,20.1797,#B3E3F5,NaN,NaN,69.0,NaN,154.0,...,atomic,25.0,27.0,0.00000,NaN,1898,18,2,NaN,NaN


In [80]:
atomic_num_concat.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,bonding type,melting point,boiling point,density,metal,year discovered,group,period,counts,Count
0,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,metallic,454.0,1615.0,0.54,alkali metal,1817,1,2,NaN,7
1,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,covalent network,2348.0,4273.0,2.46,metalloid,1807,13,2,NaN,3
2,8,O,Oxygen,15.9994,#FF0D0D,[He] 2s2 2p4,3.44,73.0,140 (-2),152.0,...,diatomic,55.0,90.0,0.00,nonmetal,1774,16,2,0.0,260
3,11,Na,Sodium,22.98976928,#AB5CF2,[Ne] 3s1,0.93,154.0,102 (+1),227.0,...,metallic,371.0,1156.0,0.97,alkali metal,1807,1,3,NaN,30
4,12,Mg,Magnesium,24.3050,#8AFF00,[Ne] 3s2,1.31,130.0,72 (+2),173.0,...,metallic,923.0,1363.0,1.74,alkaline earth metal,1808,2,3,NaN,57


In [94]:
atomic_num_concat['Count'].fillna(value=0, inplace=True)

In [98]:
atomic_num_concat.loc[atomic_num_concat['symbol'] == 'O', 'Count'] = 0

In [99]:
print(atomic_num_concat.loc[atomic_num_concat['symbol'] == 'O'])

   atomic number symbol    name atomic mass      CPK electronic configuration  \
7              8      O  Oxygen     15.9994  #FF0D0D             [He] 2s2 2p4   

   electronegativity  atomic radius ion radius  van der Waals radius  ...    \
7               3.44           73.0   140 (-2)                 152.0  ...     

   melting point  boiling point density     metal  year discovered  group  \
7           55.0           90.0     0.0  nonmetal             1774     16   

   period counts Count Counts  
7       2    0.0   0.0    0.0  

[1 rows x 24 columns]


In [101]:
output_file("ellen_periodic.html")

periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = atomic_num_concat.copy()
df["atomic mass"] = df["atomic mass"].astype(str)
df["group"] = df["group"].astype(str)
df["period"] = [periods[x-1] for x in df.period]
df = df[df.group != "-"]
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]

# cmap = {
#     "alkali metal"         : "#a6cee3",
#     "alkaline earth metal" : "#1f78b4",
#     "metal"                : "#d93b43",
#     "halogen"              : "#999d9a",
#     "metalloid"            : "#e08d49",
#     "noble gas"            : "#eaeaea",
#     "nonmetal"             : "#f1d4Af",
#     "transition metal"     : "#599d7A",
# }

# colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
colors = RdBu[11]
mapper = LinearColorMapper(palette=colors, low=df['Count'].min(), high=df['Count'].max())

source = ColumnDataSource(df)

p = figure(title="Periodic Table (omitting LA and AC Series)", plot_width=1000, plot_height=450,
           tools="", toolbar_location=None,
           x_range=groups, y_range=list(reversed(periods)))

p.rect("group", "period", 0.95, 0.95, source=source, fill_alpha=0.6, legend="metal",
       fill_color=transform('Count', mapper))
       #color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

text_props = {"source": source, "text_align": "left", "text_baseline": "middle"}

x = dodge("group", -0.4, range=p.x_range)

r = p.text(x=x, y="period", text="symbol", **text_props)
r.glyph.text_font_style="bold"

r = p.text(x=x, y=dodge("period", 0.3, range=p.y_range), text="atomic number", **text_props)
r.glyph.text_font_size="8pt"

r = p.text(x=x, y=dodge("period", -0.35, range=p.y_range), text="name", **text_props)
r.glyph.text_font_size="5pt"

r = p.text(x=x, y=dodge("period", -0.2, range=p.y_range), text="atomic mass", **text_props)
r.glyph.text_font_size="5pt"

p.text(x=["3", "3"], y=["VI", "VII"], text=["LA", "AC"], text_align="center", text_baseline="middle")

p.add_tools(HoverTool(tooltips = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass}"),
    ("Type", "@metal"),
    ("CPK color", "$color[hex, swatch]:CPK"),
    ("Electronic configuration", "@{electronic configuration}"),
]))

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location ="top_center"

show(p)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='94ccf16d-eb6a-4dd5-982e-cf38783d290a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='3860c967-f9e1-4ff3-8f3b-ec96e68a095a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='ada4e548-d29d-41f2-b60a-38fd04fa7529', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='f9ffe146-7487-408a-b45a-7958306272f0', ...)]
/Users/Ellen_M/miniconda3/lib/python3.6/site-packages/bokeh/core/json_encoder.py:80: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.



In [117]:
output_file("ellen_periodic.html")

periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = atomic_num_concat.copy()
df["atomic mass"] = df["atomic mass"].astype(str)
df["group"] = df["group"].astype(str)
df["period"] = [periods[x-1] for x in df.period]
df = df[df.group != "-"]
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]
colors = RdBu[11]
source = ColumnDataSource(df)

def make_plot(parameter):
    mapper = LinearColorMapper(palette=colors, low=df[parameter].min(), high=df[parameter].max())



    plot = figure(title="Periodic Table (omitting LA and AC Series)", plot_width=1000, plot_height=450,
           tools="", toolbar_location=None,
           x_range=groups, y_range=list(reversed(periods)))

    plot.rect("group", "period", 0.95, 0.95, source=source, fill_alpha=0.6, legend="metal",
       fill_color=transform(parameter, mapper))
       #color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

    text_props = {"source": source, "text_align": "left", "text_baseline": "middle"}

    x = dodge("group", -0.4, range=p.x_range)

    r = plot.text(x=x, y="period", text="symbol", **text_props)
    r.glyph.text_font_style="bold"

    r = plot.text(x=x, y=dodge("period", 0.3, range=p.y_range), text="atomic number", **text_props)
    r.glyph.text_font_size="8pt"

    r = plot.text(x=x, y=dodge("period", -0.35, range=p.y_range), text="name", **text_props)
    r.glyph.text_font_size="5pt"

    r = plot.text(x=x, y=dodge("period", -0.2, range=p.y_range), text="atomic mass", **text_props)
    r.glyph.text_font_size="5pt"

    plot.text(x=["3", "3"], y=["VI", "VII"], text=["LA", "AC"], text_align="center", text_baseline="middle")

    plot.add_tools(HoverTool(tooltips = [
        ("Name", "@name"),
        ("Atomic number", "@{atomic number}"),
        ("Atomic mass", "@{atomic mass}"),
        ("Type", "@metal"),
        ("CPK color", "$color[hex, swatch]:CPK"),
        ("Electronic configuration", "@{electronic configuration}"),
    ]))

    plot.outline_line_color = None
    plot.grid.grid_line_color = None
    plot.axis.axis_line_color = None
    plot.axis.major_tick_line_color = None
    plot.axis.major_label_standoff = 0
    plot.legend.orientation = "horizontal"
    plot.legend.location ="top_center"

    return plot

def update_plot(attrname, old, new):
    parameter = parameter_select.value
    make_plot(parameter)

parameter = parameter_select.value
    
parameter_select = Select(title="Parameter:", value='electronegativity', options=['electronegativity', 'atomic radius', 'melting point'])
  
parameter_select.on_change('value', update_plot)

my_plot = make_plot(parameter)

show(layout([[my_plot],[parameter_select]]))

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='94ccf16d-eb6a-4dd5-982e-cf38783d290a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='3860c967-f9e1-4ff3-8f3b-ec96e68a095a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='ada4e548-d29d-41f2-b60a-38fd04fa7529', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='f9ffe146-7487-408a-b45a-7958306272f0', ...)]
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='b664b0b6-2e22-4732-abe7-33241858e365', ...)
/Users/Ellen_M/miniconda3/lib/python3.6/site-packages/bokeh/core/json_encoder.py:80: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).typ

In [111]:
my_plot = make_plot('electronegativity')
show(my_plot)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='94ccf16d-eb6a-4dd5-982e-cf38783d290a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='3860c967-f9e1-4ff3-8f3b-ec96e68a095a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='ada4e548-d29d-41f2-b60a-38fd04fa7529', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: Count [renderer: GlyphRenderer(id='f9ffe146-7487-408a-b45a-7958306272f0', ...)]
W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure(id='b664b0b6-2e22-4732-abe7-33241858e365', ...)
/Users/Ellen_M/miniconda3/lib/python3.6/site-packages/bokeh/core/json_encoder.py:80: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).typ